In [27]:
#Importar librerías
import pandas as pd
import ast

In [28]:
# Importar datasets
movies = pd.read_csv('Dataset/movies_dataset.csv')
credits = pd.read_csv('Dataset/credits.csv')

C:\Users\crest\AppData\Local\Temp\ipykernel_16780\2056344466.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('Dataset/movies_dataset.csv')


In [29]:
# Eliminar filas duplicadas por id en movies
movies_sin_duplicados = movies.drop_duplicates(subset=['id'])

# Eliminar filas duplicadas por id en credits
credits_sin_duplicados = credits.drop_duplicates(subset=['id'])

# Corroborar un menor número de filas resultantes en cada dataset
print("Número de filas en movies_dataset sin duplicados:", movies_sin_duplicados.shape[0])
print("Número de filas en credits sin duplicados:", credits_sin_duplicados.shape[0])

Número de filas en movies_dataset sin duplicados: 45436
Número de filas en credits sin duplicados: 45432


* Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [30]:
# Desanidar datos de 'movies_dataset.csv'
columns_to_unnest = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
# Reemplazar NaN por strings vacíos
movies[columns_to_unnest] = movies[columns_to_unnest].fillna('')
# Función para evaluar los valores de la columna
def evaluate_value(value):
    try:
        return ast.literal_eval(value)
    except (SyntaxError, ValueError):
        return value
# Iterar columnas y aplicar función de evaluación
for column in columns_to_unnest:
    movies[column] = movies[column].apply(evaluate_value)

In [31]:
# Desanidar datos de 'credits.csv'
columns_to_unnest = ['cast', 'crew', 'id']
# Reemplazar NaN por strings vacíos
credits[columns_to_unnest] = credits[columns_to_unnest].fillna('')
# Función para evaluar los valores de la columna
def evaluate_value(value):
    try:
        return ast.literal_eval(value)
    except (SyntaxError, ValueError):
        return value
# Iterar columnas y aplicar función de evaluación
for column in columns_to_unnest:
    credits[column] = credits[column].apply(evaluate_value)

* Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [32]:
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

* Los valores nulos del campo release date deben eliminarse.

In [33]:
movies.dropna(subset=['release_date'], inplace=True)

* De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [34]:
# Convertir filas de "release_year" al formato deseado, y eliminar las filas con valores inválidos.
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')
movies = movies[movies['release_date'].notna()]

# Crear la columna 'release_year'
movies['release_year'] = movies['release_date'].dt.year

# Verificar cambios
print(movies[['release_date', 'release_year']].head())

  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
3   1995-12-22          1995
4   1995-02-10          1995


* Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [35]:
# Convertir columnas "revenue" y "budget" a numéricas
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')
# Calcular el retorno de inversión como "return"
movies['return'] = movies['revenue'] / movies['budget']
movies['return'].fillna(0, inplace=True)
# Verificar cambios
print(movies['return'].head())

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4          inf
Name: return, dtype: float64


* Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [36]:
columns_to_drop = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"]
movies.drop(columns=columns_to_drop, inplace=True)

In [37]:
print(movies['genres'].head())

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object


Crear columna "director" a partir de credits.csv en movies_dataset.csv

In [49]:
import pandas as pd
import re

patron = r"'Director', 'name': '(,*?)', "

def extraer_nombres_director(valor):
    if isinstance(valor, list):
        nombres_director = []
        for diccionario in valor:
            if diccionario.get("job") == "Director":
                nombres_director.append(diccionario.get("name"))
        return nombres_director
    else:
        resultado = re.findall(patron, valor)
        if resultado:
            return resultado
        else:
            return ""

# Convertir la columna "id" al mismo tipo de dato en ambos datasets
movies["id"] = movies["id"].astype(int)
credits["id"] = credits["id"].astype(int)

# Fusionar los datasets utilizando el sufijo "_credits" para evitar colisión de nombres
df_merged = pd.merge(movies, credits, on="id", suffixes=("", "_credits"))

# Aplicar la función para extraer el nombre del director
df_merged["director"] = df_merged["crew"].apply(extraer_nombres_director)

# Imprimir la columna "director"
print(df_merged["director"])

0           [John Lasseter]
1            [Joe Johnston]
2           [Howard Deutch]
3         [Forest Whitaker]
4           [Charles Shyer]
                ...        
45446          [John Irvin]
45447            [Lav Diaz]
45448      [Mark L. Lester]
45449    [Yakov Protazanov]
45450       [Daisy Asquith]
Name: director, Length: 45451, dtype: object


In [50]:
# Eliminar columnas innecesarias
df_merged = df_merged.drop('cast', axis=1)
df_merged = df_merged.drop('crew', axis=1)

In [51]:
print(df_merged.columns)

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'director'],
      dtype='object')


Guardar todas las transformaciones en un nuevo dataset, para ser consumido más eficientemente por la API

In [52]:
df_merged.to_csv('movies_dataset_clean.csv', index=False)